Strategy:
new range definition with fibonacci extensions - the basic idea is that when a range has been broken out of, it makes sense to target the span of that range * 1.618. still working on it but maybe i could chart out what i’m trying so i can see visually whether it works or not. what i’m thinking is that a range could be identified when the highest high and lowest low haven’t changed over 50% or 75% of the lookback period or whatever, because in theory the range holds more weight if it has lasted a longer time. This is like a filter to stop the feature from constantly calling a range breakout during trending periods, and since the fibonacci part of the signal relies quite heavily on the levels of the range being significant, it’s important to try to filter out all the meaningless range breakouts. so i need a vectorised way to keep track of when the hh and ll changes, and keep a constant count of how long it has been since the last change. i could refine it even further by saying if the range only changed by a small amount (maybe 1%-5% of the total range?) and has since continued to be a range, then it’s just a deviation and therefore doesn’t count. so that logic could look like: if all highest highs over the most recent 50% of the lookback period are within 5% of each other, the range high is the mean of those values and a break of the highest of those would be considered a resistance break. if highs and lows conform to these rules then there is a range in effect and a break of either of these levels is considered a range break.

Technical method used to quantify whether stop or target was hit first:
I want to find the index of the first high / low to exceed the profit value and the index of the first low / high to exceed the stop value, then I can see which is first. I can use idxmax and idxmin for this, but I first need to use clip to make sure that the first values to exceed my limits will be considered the first min/max value

i have a few strategies i could try out here:
- if support or resistance is broken whilst 'since_broke_sup' or 'since_broke_res' is above the 'split' threshold, expect continuation in the direction of the break. The approach for this strategy would be to place an oco order on the orderbook at the support/resistance line as soon as the relevant count goes above the split threshold
- if support or resistance is broken whilst the relevant count is below the 'split' threshold, expect rejection/reversal back into/across the channel. the way to take these trades would be to wait for support/resistance to be broken, then open a trade with an oco order on the close of the candle that broke it.
- if price bounces near support or resistance without actually breaking it, that might suggest a reversal too. i'll have to work out if it's a high probability signal, but it certainly gives clear invalidation. these trades would also be taken at market on any close that was close enough to the channel boundary

In [1]:
import time
import pandas as pd
from pathlib import Path
import joblib
import mt.resources.ml_funcs as mlf
import mt.resources.indicators as ind
import mt.resources.binance_funcs as funcs
import mt.resources.features as features
import numpy as np
import json
from itertools import product
from collections import Counter
from datetime import datetime
from pyarrow import ArrowInvalid
import statistics as stats
from plotly.subplots import make_subplots
import plotly.graph_objects as go
if not Path('/pi_2.txt').exists():
    from sklearnex import patch_sklearn
    patch_sklearn()
from sklearn.dummy import DummyClassifier
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.svm import SVC
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, BaggingClassifier, StackingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split, RandomizedSearchCV, cross_val_score
from sklearn.preprocessing import QuantileTransformer, MinMaxScaler
from sklearn.compose import ColumnTransformer
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, fbeta_score
from sklearn.metrics import confusion_matrix, roc_auc_score, make_scorer
from sklearn.inspection import permutation_importance
from sklearn.feature_selection import SelectKBest, mutual_info_classif, f_classif, chi2
from mlxtend.feature_selection import SequentialFeatureSelector as SFS
from imblearn.under_sampling import RandomUnderSampler, ClusterCentroids
from xgboost import XGBClassifier, DMatrix
import optuna

Intel(R) Extension for Scikit-learn* enabled (https://github.com/intel/scikit-learn-intelex)


In [2]:
def backtest_oco(df_0, side, lookback, trim_ohlc=2000):
    """i can either target the opposite side of the channel or the mid-point, or both"""

    df_0 = df_0.reset_index(drop=True)
    atr_lb = 10
    df_0 = ind.atr(df_0, atr_lb)
    
    # identify potential entries
    rows = list(df_0.loc[df_0[f"entry_{side[0]}"]].index)
    
    results = []
    for row in rows:
        if row == len(df_0)-1:
            break
        df = df_0[row:row + trim_ohlc].copy().reset_index(drop=True)
        entry = df.close.iloc[0]
        atr = df[f"atr-{atr_lb}"].iloc[0]
        
        if side == 'long':
            highest = df.high.max()
            target = df[f"hh_{lookback}"].iloc[0]
            stop = df[f"ll_{lookback}"].iloc[0] - atr
            rr = abs((target / entry) - 1) / abs((stop / entry) - 1)
            target_hit_idx = df.high.clip(upper=target).idxmax()
            stop_hit_idx = df.low.clip(lower=stop).idxmin()
            if (target > highest) or (stop_hit_idx < target_hit_idx):
                exit_row = stop_hit_idx
                pnl_cat = 0
                pnl = (stop-entry)/entry
            elif target_hit_idx < stop_hit_idx:
                exit_row = target_hit_idx
                pnl_cat = 1
                pnl = (target-entry)/entry
            else:
                exit_row = stop_hit_idx
                pnl_cat = 0
                pnl = 0
        else:
            lowest = df.low.min()
            target = df[f"ll_{lookback}"].iloc[0]
            stop = df[f"hh_{lookback}"].iloc[0] + atr
            rr = abs((target / entry) - 1) / abs((stop / entry) - 1)
            target_hit_idx = df.low.clip(lower=target).idxmin()
            stop_hit_idx = df.high.clip(upper=stop).idxmax()
            if (target < lowest) or (stop_hit_idx < target_hit_idx):
                exit_row = stop_hit_idx
                pnl_cat = 0
                pnl = (entry-stop)/entry
            elif target_hit_idx < stop_hit_idx:
                exit_row = target_hit_idx
                pnl_cat = 1
                pnl = (entry-target)/entry
            else:
                exit_row = stop_hit_idx
                pnl_cat = 0
                pnl = 0

        row_data = df_0.iloc[row-1].to_dict()
        pnl_pct = pnl - (2 * 0.0015)  # subtract trading fees and slippage estimate

        row_res = dict(
            # idx=row,
            r_pct=atr,
            rr=rr,
            lifespan=exit_row,
            pnl_pct=pnl_pct,
            pnl_r=pnl_pct / atr,
            pnl_cat=pnl_cat
        )

        results.append(row_data | row_res)

        msg = f"trade lifespans getting close to trimmed ohlc length ({exit_row / trim_ohlc:.1%}), increase trim ohlc"
        if exit_row / trim_ohlc > 0.9:
            print(msg)
        
    return results

In [3]:
def prepare_strat_data(df, lookback):    
    df[f"ll_{lookback}"] = df.low.rolling(lookback).min()
    df[f"hh_{lookback}"] = df.high.rolling(lookback).max()
    
    df['channel_mid'] = (df[f"hh_{lookback}"] + df[f"ll_{lookback}"]) / 2
    df['channel_width'] = (df[f"hh_{lookback}"] - df[f"ll_{lookback}"]) / df.channel_mid
    
    df['broke_support'] = df.low == df[f"ll_{lookback}"]
    df['broke_resistance'] = df.high == df[f"hh_{lookback}"]
    
    df['close_above_sup'] = df.close > df[f"ll_{lookback}"].shift()
    df['close_below_res'] = df.close < df[f"hh_{lookback}"].shift()
    
    df['channel_position'] = (df.close - df[f"ll_{lookback}"]) / (df[f"hh_{lookback}"] - df[f"ll_{lookback}"])
    
    df['entry_l'] = df.channel_position < 0.05
    df['entry_s'] = df.channel_position > 0.95
    
    df['entry_l_price'] = df.close.loc[df.entry_l]
    df['entry_s_price'] = df.close.loc[df.entry_s]
    
    df['support_diff_z'] = abs(ind.z_score(df[f"ll_{lookback}"].ffill().pct_change(), lookback) * df.broke_support)
    df['resistance_diff_z'] = abs(ind.z_score(df[f"hh_{lookback}"].ffill().pct_change(), lookback) * df.broke_resistance)
    
    return df

In [4]:
def plot_stuff(df):
    df = df.tail(2000)
    
    fig = make_subplots(rows=2, cols=1, shared_xaxes=True,
                        vertical_spacing=0.01,
                        row_heights=[0.8, 0.2])
    
    fig.update_yaxes(title_text="Price", row=1, col=1)
    fig.update_yaxes(title_text="S/R Diff Z-score", row=2, col=1)
    
    fig.add_trace(go.Candlestick(x=df.timestamp, open=df.open, high=df.high, low=df.low, close=df.close, showlegend=False), row=1, col=1)
    fig.add_trace(go.Scatter(x=df['timestamp'], y=df[f"hh_{lookback}"], mode='lines', name='Highest High'), row=1, col=1)
    fig.add_trace(go.Scatter(x=df['timestamp'], y=df[f"ll_{lookback}"], mode='lines', name='Lowest Low'), row=1, col=1)
    fig.add_trace(go.Scatter(x=df['timestamp'], y=df.channel_mid, mode='lines', name='Channel Mid-point'), row=1, col=1)
    fig.add_trace(go.Scatter(x=df['timestamp'], y=df.entry_l_price, mode='markers', 
                             marker_size=10, marker_symbol='triangle-up', 
                             name='Strat 2 Long Entry'), row=1, col=1)
    fig.add_trace(go.Scatter(x=df['timestamp'], y=df.entry_s_price, mode='markers', 
                             marker_size=10, marker_symbol='triangle-down', 
                             name='Strat 2 Short Entry'), row=1, col=1)
    
    fig.add_trace(go.Scatter(x=df['timestamp'], y=df[f"channel_position"], mode='lines', name='Channel Position'), row=2, col=1)
    # fig.add_trace(go.Scatter(x=df['timestamp'], y=df[f"support_diff_z"], mode='lines', name='Support Diff Z Score'), row=2, col=1)
    # fig.add_trace(go.Scatter(x=df['timestamp'], y=df[f"resistance_diff_z"], mode='lines', name='Resistance Diff Z Score'), row=2, col=1)
    fig.update_layout(height=600, width=1600, xaxis_rangeslider_visible=False,
                      margin=go.layout.Margin(l=20, r=20, b=20, t=20))
    fig.show()

In [5]:
# set constants
timeframe = '1h'
side = 'short'
# r_mult = 2
data_len = 50000
lookback = 200
selection_method = '1w_volumes'
num_pairs = 150

Create the Dataset

In [6]:
# generate dataset
pairs = mlf.get_margin_pairs(selection_method, num_pairs)
all_res = []
for pair in pairs:
    df = mlf.get_data(pair, timeframe)
    # df = df.tail(20000).reset_index(drop=True)
    df = mlf.add_features(df, timeframe)
    df = prepare_strat_data(df, lookback)
    print(f"{pair} len: {len(df)}")
    res = backtest_oco(df, side, lookback)
    all_res.extend(res)
res_df = pd.DataFrame(all_res).sort_values('timestamp').reset_index(drop=True)
res_df = res_df.dropna(axis=1)
res_df.tail()

SOLUSDT len: 8760
XRPUSDT len: 8760
FDUSDUSDT len: 2460
BTCUSDT len: 8760
BNBUSDT len: 8760
ARKUSDT len: 1072
GASUSDT len: 5606
LINKUSDT len: 8760
ETHUSDT len: 8760
DOGEUSDT len: 8760
ARBUSDT len: 5455
RUNEUSDT len: 8760
CAKEUSDT len: 8760
MATICUSDT len: 8760
SUSHIUSDT len: 8760
GALAUSDT len: 8760
TRBUSDT len: 8760
ADAUSDT len: 8760
POLYXUSDT len: 6539
AVAXUSDT len: 8760
LTCUSDT len: 8760
ATOMUSDT len: 8760
CYBERUSDT len: 1980
DOTUSDT len: 8760
SHIBUSDT len: 8760
SNTUSDT len: 4096
INJUSDT len: 8760
AAVEUSDT len: 8760
UNIUSDT len: 8760
NEARUSDT len: 8760
TRXUSDT len: 8760
PEPEUSDT len: 4422
SUIUSDT len: 4476
OPUSDT len: 6539
EGLDUSDT len: 8760
BLZUSDT len: 8760
SLPUSDT len: 8760
FETUSDT len: 8760
RNDRUSDT len: 8760
APTUSDT len: 6539
MASKUSDT len: 8760
UNFIUSDT len: 8760
NEOUSDT len: 8760
CFXUSDT len: 8760
FILUSDT len: 8760
STRAXUSDT len: 8760
WLDUSDT len: 2511
LOOMUSDT len: 5438
FTMUSDT len: 8760
BCHUSDT len: 8760
DYDXUSDT len: 8760
GRTUSDT len: 8760
OSMOUSDT len: 6539
APEUSDT len: 8760

,timestamp,open,high,low,close,base_vol,quote_vol,num_trades,taker_buy_base_vol,taker_buy_quote_vol,...,close_below_res,entry_l,entry_s,atr-10,r_pct,rr,lifespan,pnl_pct,pnl_r,pnl_cat
17953,2023-11-05 17:00:00+00:00,70.9400,71.8800,70.7800,71.5800,2.0234e+04,1.4453e+06,5494.0,1.0685e+04,7.6323e+05,...,True,False,False,0.6578,0.6525,7.0959,1,-0.0135,-0.0206,0
17954,2023-11-05 17:00:00+00:00,0.1220,0.1225,0.1214,0.1214,6.9719e+06,8.5103e+05,2222.0,4.4669e+06,5.4557e+05,...,True,False,False,0.0019,0.0019,8.8768,1,-0.0232,-12.4375,0
17955,2023-11-05 18:00:00+00:00,0.6481,0.6529,0.6473,0.6526,7.6912e+06,4.9975e+06,7685.0,3.7783e+06,2.4557e+06,...,True,False,True,0.0082,0.0084,8.3057,0,-0.0239,-2.8519,0
17956,2023-11-05 21:00:00+00:00,0.0787,0.0787,0.0772,0.0775,1.0770e+07,8.3891e+05,3793.0,5.5456e+06,4.3204e+05,...,True,False,False,0.0014,0.0015,7.9403,1,-0.0285,-19.5650,0
17957,2023-11-05 21:00:00+00:00,243.7000,243.9000,240.7000,242.0000,1.8933e+04,4.5840e+06,12384.0,9.2482e+03,2.2392e+06,...,True,False,False,2.0953,2.1593,8.0092,1,-0.0143,-0.0066,0


In [7]:
# split features from labels
X, y, _ = mlf.features_labels_split(res_df)

In [8]:
# undersampling
# us = RandomUnderSampler(random_state=0)
us = ClusterCentroids(random_state=0)
X, y = us.fit_resample(X, y)

/home/ross/coding/modular_trader/venv/lib/python3.11/site-packages/sklearn/cluster/_kmeans.py:1416: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
/home/ross/coding/modular_trader/venv/lib/python3.11/site-packages/daal4py/sklearn/cluster/_k_means_0_23.py:184: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


In [9]:
# split data for fitting and calibration
X_final = X.copy()
y_final = y.copy()
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=11, stratify=y)
X_test, X_val, y_test, y_val = train_test_split(X_test, y_test, test_size=0.5, random_state=11, stratify=y_test)

In [10]:
# feature scaling
cols = list(X_train.columns) # list of strings, names of all features
scaler = MinMaxScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)
X_val = scaler.transform(X_val)

# turn back into dataframes
X_train = pd.DataFrame(X_train, columns=cols)
X_test = pd.DataFrame(X_test, columns=cols)
X_val = pd.DataFrame(X_val, columns=cols)

In [11]:
# remove features with too many missing values
missing_condition = X_train.columns[X_train.isnull().mean(axis=0) < 0.1]
X_train = X_train[missing_condition]
X_test = X_test[missing_condition]
X_val = X_val[missing_condition]

# remove low variance features
variance_condition = X_train.columns[X_train.var() > 0.0]
X_train = X_train[variance_condition]
X_test = X_test[variance_condition]
X_val = X_val[variance_condition]


In [12]:
# remove features that are highly correlated with other features
corr_thresh = 0.5
corr_matrix = X_train.corr()
# Extract the upper triangle of the correlation matrix
upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k = 1).astype(bool))
        
# Select the features with correlations above the threshold
# Need to use the absolute value
to_drop = [column for column in upper.columns if any(upper[column].abs() > corr_thresh)]

# Dataframe to hold correlated pairs
record_collinear = []

# Iterate through the columns to drop to record pairs of correlated features
for column in to_drop:

    # Find the correlated features
    corr_features = list(upper.index[upper[column].abs() > corr_thresh])

    # Find the correlated values
    corr_values = list(upper[column][upper[column].abs() > corr_thresh])
    drop_features = [column for _ in range(len(corr_features))]    

    # Record the information (need a temp df for now)
    temp_df = pd.DataFrame.from_dict({'drop_feature': drop_features,
                                     'corr_feature': corr_features,
                                     'corr_value': corr_values})

    # Add to dataframe
    record_collinear.append(temp_df)

record_collinear = pd.concat(record_collinear, axis=0, ignore_index=True)
X_train = X_train.drop(list(record_collinear.corr_feature), axis=1)
X_test = X_test.drop(list(record_collinear.corr_feature), axis=1)
X_val = X_val.drop(list(record_collinear.corr_feature), axis=1)
X_train.shape

(772, 47)

In [13]:
# mutual info feature selection
print(f"feature selection began: {datetime.now().strftime('%Y/%m/%d %H:%M')}")
cols = list(X_train.columns) # list of strings, names of all features
selector = SelectKBest(mutual_info_classif, k=15)
selector.fit(X_train, y_train)
X_train = selector.transform(X_train)
X_test = selector.transform((X_test))
X_val = selector.transform((X_val))

# Turn data back into dataframe with column names
cols_idx = list(selector.get_support(indices=True))
selected_columns = [col for i, col in enumerate(cols) if i in cols_idx]
X_train = pd.DataFrame(X_train, columns=selected_columns)
X_test = pd.DataFrame(X_test, columns=selected_columns)
X_val = pd.DataFrame(X_val, columns=selected_columns)
print(selected_columns)

feature selection began: 2023/11/06 00:13
['ats_z_200', 'day_of_week', 'unweighted_bear_doji', 'fractal_trend_age_long', 'fractal_trend_age_short', 'hour', 'hour_180', 'spooky_num_prox', 'stoch_m_100', 'high_volume_churn_50', 'vol_delta', 'vol_delta_pct', 'weekly_open_ratio', 'broke_resistance', 'rr']


In [14]:
# sequential feature selection
pre_selector_model = RandomForestClassifier()
selector = SFS(estimator=pre_selector_model, k_features='best', forward=False,
                     floating=True, verbose=0, scoring='accuracy', n_jobs=-1)
selector = selector.fit(X_train, y_train)
cols_idx = list(selector.k_feature_idx_)
X_train = selector.transform(X_train)
X_test = selector.transform(X_test)
X_val = selector.transform(X_val)

RandomForestClassifierHyperparameter Optimisation

In [15]:
def objective(trial):
    # Suggest values for hyperparameters
    # criterion = trial.suggest_categorical('criterion', 'gini', 'log_loss')
    # min_samples_split = trial.suggest_int("min_samples_split", 2, 10)
    n_estimators = trial.suggest_int("n_estimators", 50, 150)
    max_depth = trial.suggest_int("max_depth", 12, 32)
    max_features = trial.suggest_float("max_features", 0.1, 1.0)
    max_samples = trial.suggest_float('max_samples', 0.1, 1.0)
    ccp_alpha = trial.suggest_float('ccp_alpha', 1e-5, 1e-2, log=True)

    # Create and fit random forest model
    model = RandomForestClassifier(
        n_estimators=n_estimators,
        criterion='log_loss',
        max_depth=max_depth,
        min_samples_split=2,
        max_features=max_features,
        max_samples=max_samples,
        ccp_alpha=ccp_alpha,
        random_state=42,
        n_jobs=-1
    )
    model.fit(X_train, y_train)

    # Score model
    scores = cross_val_score(model, X_test, y_test, n_jobs=-1)
    avg_score = stats.mean(scores)

    # Return score
    return avg_score

study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=1000, n_jobs=-1)

# automatically get best params from study
best_trials = [trial.params for trial in study.trials if trial.values[0] >= (study.best_value - 0.001)]
best_df = pd.DataFrame(best_trials)
best_df.describe()
best_params = best_df.median(axis=0).to_dict()

[I 2023-10-18 00:01:13,686] A new study created in memory with name: no-name-e67fc058-44e4-4b4d-ba6f-16e2175626dc
[I 2023-10-18 00:01:15,317] Trial 5 finished with value: 0.9322222222222222 and parameters: {'n_estimators': 58, 'max_depth': 22, 'max_features': 0.5864997308530078, 'max_samples': 0.23281933045408443, 'ccp_alpha': 3.849725038718405e-05}. Best is trial 5 with value: 0.9322222222222222.
[I 2023-10-18 00:01:15,652] Trial 4 finished with value: 0.932020202020202 and parameters: {'n_estimators': 61, 'max_depth': 19, 'max_features': 0.924105915092956, 'max_samples': 0.1613355432382979, 'ccp_alpha': 0.00038868655239946646}. Best is trial 5 with value: 0.9322222222222222.
[I 2023-10-18 00:01:16,163] Trial 0 finished with value: 0.9319191919191919 and parameters: {'n_estimators': 83, 'max_depth': 25, 'max_features': 0.5768542889754996, 'max_samples': 0.5618942397546192, 'ccp_alpha': 0.00041890340223929676}. Best is trial 5 with value: 0.9322222222222222.
[I 2023-10-18 00:01:16,746]

In [16]:
# remove features with low permutation importance
imp_model = RandomForestClassifier()
imp_model.fit(X_train, y_train)
# print(f"Score: {imp_model.score(X_test, y_test)}")
importances = permutation_importance(imp_model, X_test, y_test, n_repeats=1000, random_state=42, n_jobs=-1)
imp_mean = pd.Series(importances.importances_mean, index=selector.k_feature_names_)
imp_std = pd.Series(importances.importances_std, index=selector.k_feature_names_)
imp_df = pd.DataFrame({'importances': imp_mean, 'imp_std': imp_std}).sort_values('importances', ascending=False)
imp_df['cum_imp'] = imp_df.importances.cumsum()
final_features = list(imp_mean.index[imp_mean > 0.01])
print(final_features)
# print(imp_df)

['unweighted_bull_doji', 'rsi_100_above_70', 'stoch_vwma_ratio_100', 'vol_delta', 'vol_delta_pct']


In [17]:
# final validation score before training production model
X_train = pd.DataFrame(X_train, columns=selector.k_feature_names_)
X_val = pd.DataFrame(X_val, columns=selector.k_feature_names_)
X_train = X_train[final_features]
X_val = X_val[final_features]
val_model = RandomForestClassifier(
    n_estimators=int(best_params['n_estimators']),
    criterion='log_loss',
    max_depth=int(best_params['max_depth']),
    min_samples_split=2,
    max_features=best_params['max_features'],
    max_samples=best_params['max_samples'],
    ccp_alpha=best_params['ccp_alpha'],
    random_state=42,
    n_jobs=-1
)
val_model.fit(X_train, y_train)
score = val_model.score(X_val, y_val)
print(f"Final model validation score: {score:.1%}")

Final model validation score: 97.3%


In [18]:
X_final = X_final[final_features]
scaler = MinMaxScaler()
X_final = scaler.fit_transform(X_final)
X_final = pd.DataFrame(X_final, columns=final_features)

final_model = RandomForestClassifier(
    n_estimators=int(best_params['n_estimators']),
    criterion='log_loss',
    max_depth=int(best_params['max_depth']),
    min_samples_split=2,
    max_features=best_params['max_features'],
    max_samples=best_params['max_samples'],
    ccp_alpha=best_params['ccp_alpha'],
    random_state=42,
    n_jobs=-1
)

final_model.fit(X_final, y_final)

RandomForestClassifier(ccp_alpha=0.00027167533107655016, criterion='log_loss',
                       max_depth=31, max_features=0.835956012426688,
                       max_samples=0.14808803110336655, n_estimators=54,
                       n_jobs=-1, random_state=42)

In [19]:
# save models and info
mlf.save_models(
    "channel_run",
    f"{lookback}",
    selection_method,
    num_pairs,
    side,
    timeframe,
    data_len,
    final_features,
    pairs,
    final_model,
    scaler,
    len(X_final)
)

In [30]:
final_model.predict_proba(X_final.iloc[-50:, :])

array([[0.        , 1.        ],
       [0.        , 1.        ],
       [0.        , 1.        ],
       [0.        , 1.        ],
       [0.01851852, 0.98148148],
       [0.        , 1.        ],
       [0.        , 1.        ],
       [0.        , 1.        ],
       [0.38888889, 0.61111111],
       [0.        , 1.        ],
       [0.        , 1.        ],
       [0.01851852, 0.98148148],
       [0.88888889, 0.11111111],
       [0.09259259, 0.90740741],
       [0.01851852, 0.98148148],
       [0.05555556, 0.94444444],
       [0.44444444, 0.55555556],
       [0.01851852, 0.98148148],
       [0.12962963, 0.87037037],
       [0.03703704, 0.96296296],
       [0.2037037 , 0.7962963 ],
       [0.        , 1.        ],
       [0.        , 1.        ],
       [0.        , 1.        ],
       [0.        , 1.        ],
       [0.        , 1.        ],
       [0.        , 1.        ],
       [0.        , 1.        ],
       [0.        , 1.        ],
       [0.2962963 , 0.7037037 ],
       [0.